In [ ]:
list_of_content_domains = ['www.google.com', 'www.facebook.com', 'www.twitter.com']
number_of_days_prior = 1
purchase_qualifier = 'AND domain LIKE "%amazon.com%'
list_of_purchases = cleaned_data[0:100]

contentdomains = {x:None for x in list_of_content_domains}


user_ids = list_of_purchases['user_id'].unique()  

for domain in list_of_content_domains:

    grand_mean = datetime.timedelta(seconds=0)
        
    for user in user_ids:

        time_domain=dict()
        fre_domain=dict()
        user_times = {x:None for x in contentdomains.keys()}
        user_purchases = list_of_purchases.loc[list_of_purchases['user_id']==user]

        for created_date in user_purchases['created_date']:
                    
            last_x_query = '''SELECT id, user_id, url, created_date, title FROM e_url_map WHERE user_id = {0}\
            AND created_date BETWEEN '{1}' - INTERVAL 7 DAY AND '{1}' \
            AND domain LIKE '%{2}%' ORDER BY created_date DESC  '''\
            .format(user,created_date,domain)
            cursor.execute(last_x_query)
            last_x_days = pd.DataFrame(cursor.fetchall(), columns = ('id', 'user_id', 'url', 'created_date', 'title'))
            if len(last_x_days)>0:
                
                TIMEDI=created_date - last_x_days['created_date'][0]
                
                if domain in time_domain:
                    time_domain[domain]=time_domain[domain]+TIMEDI
                    fre_domain[domain]=fre_domain[domain]+1
                else:                   
                    time_domain[domain]=TIMEDI
                    fre_domain[domain]=1
            else:
                pass
        try:
            user_ave=time_domain[domain]/fre_domain[domain]
            grand_mean = grand_mean + user_ave
        except:
            pass
    grand_mean = grand_mean/len(user_ids)
    print( domain, grand_mean)

In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "import mysql.connector\n",
    "import pandas as pd\n",
    "import time\n",
    "import datetime"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 147,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "cnx = mysql.connector.connect(user='openupadmin', password='6C7E2F760ABFF371',\n",
    "                              host='openupdb.cxzofl0nts6g.us-east-1.rds.amazonaws.com', port = 3306,\n",
    "                              database='Openup_db')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 148,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "cursor = cnx.cursor()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 92,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "cursor.execute(query)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 93,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "data = pd.DataFrame(cursor.fetchall())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# All user purchase data, return cleaned data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 162,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": [
    "query = \"SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE has_thank_you_confirmation = 1 AND is_beauty=1 AND user_id > 50001\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 163,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "cursor.execute(query)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 164,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "data = pd.DataFrame(cursor.fetchall(),columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 165,
   "metadata": {
    "collapsed": false
   },
   "outputs": [],
   "source": [
    "# this function takes a series of PURCHASE TIMES and returns the indices of urls that are no less than delta \n",
    "# minutes apart\n",
    "\n",
    "def clean_checkout_repeats(userdata, delta):\n",
    "    cleaned_indices = []\n",
    "    user_index = list(userdata.index)\n",
    "    for count, index in enumerate(user_index):\n",
    "        if user_index[count] > -1 and user_index[count] < max(user_index):\n",
    "            if (userdata['created_date'][user_index[count]] + datetime.timedelta(minutes = delta) < userdata['created_date'][user_index[count + 1]])\\\n",
    "            or (userdata['domain'][user_index[count]] != userdata['domain'][user_index[count+1]]):\n",
    "                cleaned_indices.append(user_index[count])\n",
    "                \n",
    "    return userdata.loc[cleaned_indices,:]\n",
    "    #return indices where row + 1 and -1 > delta"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 166,
   "metadata": {
    "collapsed": false,
    "scrolled": false
   },
   "outputs": [],
   "source": [
    "cleaned_data = pd.DataFrame(columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date']) \n",
    "     \n",
    "for user in data['user_id'].unique():\n",
    "    cleaned_data = pd.concat((cleaned_data,clean_checkout_repeats(data.loc[data['user_id']==user], 60)))\n",
    "    "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 167,
   "metadata": {
    "collapsed": false
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "461"
      ]
     },
     "execution_count": 167,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(cleaned_data['user_id'].unique())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {
    "collapsed": true
   },
   "source": [
    "# Function to determine average time between domain and purchase"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# version 2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 171,
   "metadata": {
    "collapsed": false
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "www.yahoo.com 0 days 04:52:07.125000\n",
      "10.140075206756592\n",
      "www.huffingtonpost.com 1 days 12:54:31.072000\n",
      "20.074867010116577\n",
      "www.refinery29.com 1 days 05:06:22.922872\n",
      "30.26970911026001\n",
      "www.cnn.com 0 days 14:41:31.779761\n",
      "40.191296339035034\n",
      "www.elle.com 0 days 17:45:52.750000\n",
      "50.516502141952515\n",
      "www.cosmopolitan.com 0:00:00\n",
      "60.728652238845825\n"
     ]
    }
   ],
   "source": [
    "tic = time.time()\n",
    "\n",
    "list_of_content_domains = ['www.yahoo.com', 'www.huffingtonpost.com', 'www.refinery29.com', 'www.cnn.com', 'www.elle.com'\\\n",
    "                           , 'www.cosmopolitan.com']\n",
    "number_of_days_prior = 1\n",
    "purchase_qualifier = 'AND domain LIKE \"%amazon.com%'\n",
    "list_of_purchases = cleaned_data[0:200]\n",
    "\n",
    "contentdomains = {x:None for x in list_of_content_domains}\n",
    "\n",
    "\n",
    "user_ids = list_of_purchases['user_id'].unique()  \n",
    "\n",
    "for domain in list_of_content_domains:\n",
    "\n",
    "    grand_mean = datetime.timedelta(seconds=0)\n",
    "        \n",
    "    for user in user_ids:\n",
    "\n",
    "        time_domain=dict()\n",
    "        fre_domain=dict()\n",
    "        user_times = {x:None for x in contentdomains.keys()}\n",
    "        user_purchases = list_of_purchases.loc[list_of_purchases['user_id']==user]\n",
    "\n",
    "        for created_date in user_purchases['created_date']:\n",
    "                    \n",
    "            last_x_query = '''SELECT id, user_id, url, created_date, title FROM e_url_map WHERE user_id = {0}\\\n",
    "            AND created_date BETWEEN '{1}' - INTERVAL 7 DAY AND '{1}' \\\n",
    "            AND domain LIKE '%{2}%' ORDER BY created_date DESC  '''\\\n",
    "            .format(user,created_date,domain)\n",
    "            cursor.execute(last_x_query)\n",
    "            last_x_days = pd.DataFrame(cursor.fetchall(), columns = ('id', 'user_id', 'url', 'created_date', 'title'))\n",
    "            if len(last_x_days)>0:\n",
    "                \n",
    "                TIMEDI=created_date - last_x_days['created_date'][0]\n",
    "                \n",
    "                if domain in time_domain:\n",
    "                    time_domain[domain]=time_domain[domain]+TIMEDI\n",
    "                    fre_domain[domain]=fre_domain[domain]+1\n",
    "                else:                   \n",
    "                    time_domain[domain]=TIMEDI\n",
    "                    fre_domain[domain]=1\n",
    "            else:\n",
    "                pass\n",
    "        try:\n",
    "            user_ave=time_domain[domain]/fre_domain[domain]\n",
    "            grand_mean = grand_mean + user_ave\n",
    "        except:\n",
    "            pass\n",
    "    grand_mean = grand_mean/len(user_ids)\n",
    "    print( domain, grand_mean)\n",
    "    \n",
    "    toc = time.time()-tic\n",
    "    print(toc)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "collapsed": true
   },
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.5.2"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}